In [1]:
# IMPORTS
import typing as tp
import discogs_client as DiscogsClient
from rapidfuzz import process as r_process, fuzz as r_fuzz
from rich import print as rprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import json
import pathlib
import dotenv
env_file_name = '/../../../.env'
env_path = pathlib.Path(os.getcwd() + env_file_name)
if env_path.exists():
    dotenv.load_dotenv(env_path, override=True)
else:
    raise FileExistsError('env file does not exist')

SPOTIPY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
DISCOGS_USER_TOKEN = os.getenv('DISCOGS_USER_TOKEN')

discogs_client = DiscogsClient.Client('ExampleApplication/0.1', user_token=DISCOGS_USER_TOKEN)
spotify_auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
spotify_client = spotipy.Spotify(client_credentials_manager=spotify_auth_manager)



# DISCOGS

# SPOTIFY

# YOUTUBE

## Setup

In [ ]:
# Setup
# This does not work for some reason - the device connection thingy requires identifying the device
# Much check how to avoid that.

# from ytmusicapi import YTMusic, OAuthCredentials
# YTMUSIC_REFRESH_TOKEN = os.getenv('YTMUSIC_REFRESH_TOKEN')
# YTMUSIC_ACCESS_TOKEN = os.getenv('YTMUSIC_ACCESS_TOKEN')
# YTMUSIC_CLIENT_ID = os.getenv('YTMUSIC_CLIENT_ID')
# YTMUSIC_CLIENT_SECRET = os.getenv('YTMUSIC_CLIENT_SECRET')
# with open('../../oauth.json') as file:
#     oauth = json.load(file)
# oauth['access_token'] = YTMUSIC_ACCESS_TOKEN
# oauth['refresh_token'] = YTMUSIC_REFRESH_TOKEN
# oauth_credentials = OAuthCredentials(client_id=YTMUSIC_CLIENT_ID, client_secret=YTMUSIC_CLIENT_SECRET)
# yt = YTMusic(oauth, oauth_credentials=oauth_credentials)

# Alt-authorisation
# ytmusicapi.setup()

In [2]:
# Alternative
# Use the API unauthorised

from ytmusicapi import YTMusic
youtube_client = YTMusic()


## Important fields

### search

In [5]:
query: str = "bind, torture, kill"
filter: tp.Literal["artists", "albums", "songs", "playlists"] = "songs"
limit: int = 10

youtube_client.search(query=query, filter=filter, limit=limit, ignore_spelling=False)

thumbnails = (
    [
        {
            "url": "https://lh3.googleusercontent.com/QvN2GI_GwaCFoZ1UuKVpAeOccis7Q02Lc5E5XRJ9l4rYxUa8bkmPEsJAhlqLxzkJWRFnNN7iX7-1Gdw=w60-h60-l90-rj",
            "width": 60,
            "height": 60,
        },
    ],
)


# ARTISTS
search_artists_results = [
    {
        "category": "Artists",
        "resultType": "artist",
        "artist": "Suicide Commando",
        "browseId": "UCrsvJ1npu-ox0CnncGhq2QA",
        "thumbnails": thumbnails,
    }
]

# ALBUMS
search_albums_results = [
    {
        "category": "Albums",
        "resultType": "album",
        "title": "Bind, Torture, Kill",
        "type": "Album",
        "year": "2006",
        "artists": [{"name": "Suicide Commando", "id": "UCrsvJ1npu-ox0CnncGhq2QA"}],  # artist - browseId
        "browseId": "MPREb_pRY2rME882K",
    },
]

# SONGS
search_songs_results = [
    {
        "category": "Songs",
        "resultType": "song",
        "title": "Bind, Torture, Kill",
        "album": {"name": "Bind, Torture, Kill", "id": "MPREb_pRY2rME882K"},  # album - browseId
        "videoId": "0W2JDPCnoUo",  # watch parameter music.youtube.com/watch?v=...
        "artists": [{"name": "Suicide Commando", "id": "UCrsvJ1npu-ox0CnncGhq2QA"}],  # artist - browseId
    },
]

### get_artist

In [6]:
artist_browse_id = search_artists_results[0]['browseId']
get_artist_result = youtube_client.get_artist(artist_browse_id)

# For songs/videos, pass the browseId to get_playlist().
# For albums/singles/shows, pass browseId and params to get_artist_albums().
get_artist_result_ = {
    "name": "Suicide Commando",
    "channelId": "UCrsvJ1npu-ox0CnncGhq2QA",
    "thumbnails": thumbnails,
    "songs": {
        "browseId": "VLOLAK5uy_l5rAodr7oluhYHTBYMamfhnwEeE4rmhKo",  # playlist?list=[2:]
        "results": [],  # first 5 songs shown on band page
    },
    "albums": {
        "browseId": "MPADUCrsvJ1npu-ox0CnncGhq2QA",  # use in get_artist_albums()
        "params": "ggMIegYIARoCAQI%3D",  # use in get_artist_albums()
        "results": [],  # first 10 albums shown on band page
    },
    "singles": {
        "browseId": "MPADUCrsvJ1npu-ox0CnncGhq2QA",  # use in get_artist_albums()
        "params": "ggMIegYIAhoCAQI%3D",  # use in get_artist_albums()
        "results": [],  # first 10 albums shown on band page
    },
    "videos": {
        "browseId": "VLOLAK5uy_mc_gtvS2jKs_NIeGYiCFl5PyLQN5FoMX8",  # playlist?list=[2:]
        "params": "ggMCCAI%3D",
        "results": [],  # first 10 albums shown on band page
    },
    "related": {
        "browseId": None,
        "results": [],  # 10 band recommendations
    },
}


### get_album

In [7]:
album_browse_id = search_albums_results[0]['browseId']
get_album_result = youtube_client.get_album(album_browse_id)

get_album_result_ = {
    "title": "Bind, Torture, Kill",
    "type": "Album",
    "thumbnails": thumbnails,
    "artists": [{"name": "Suicide Commando", "id": "UCrsvJ1npu-ox0CnncGhq2QA"}],  # get_artist_result_['channelId']
    "year": "2006",
    "audioPlaylistId": "OLAK5uy_klMA5C-aUaXA50Ts7QAnvSFve6PMJCOww",  # album id
    "tracks": [
        {
            "videoId": "0W2JDPCnoUo",  # watch?v=[:]&list=['audioPlaylistId]
            "title": "Bind, Torture, Kill",
            "artists": [{"name": "Suicide Commando", "id": "UCrsvJ1npu-ox0CnncGhq2QA"}],  # get_artist_result_['channelId']
            "album": "Bind, Torture, Kill",
            "videoType": "MUSIC_VIDEO_TYPE_ATV",
            "trackNumber": 1,
        }, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, 
    ],
}


### get_song

In [8]:
song_video_id = search_songs_results[0]['videoId']
get_song_result = youtube_client.get_song(song_video_id)

get_song_result_ = {
    "videoDetails": {
        "videoId": "0W2JDPCnoUo",
        "title": "Bind, Torture, Kill",
        "channelId": "UCrsvJ1npu-ox0CnncGhq2QA",  # artist id
        "thumbnail": {
            "thumbnails": thumbnails
        },
        "author": "Suicide Commando",
        "musicVideoType": "MUSIC_VIDEO_TYPE_ATV",
    },
}

embed_frame = f"""
<iframe
    width="560"
    height="315"
    src="https://www.youtube.com/embed/{song_video_id}"
    title="YouTube video player"
    frameborder="0"
    allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share"
    referrerpolicy="strict-origin-when-cross-origin"
    allowfullscreen></iframe>
"""

### get_artist_albums

In [16]:
channelId = get_artist_result["albums"]["browseId"]   # 'albums' or 'singles'
params = get_artist_result["albums"]["params"]

get_artist_albums_results = youtube_client.get_artist_albums(channelId, params)

get_artist_albums_results_ = [
    {
        "browseId": "MPREb_ag6aoNYIkLi",  # album id
        "playlistId": "OLAK5uy_nhKDYPycY8VTU5vCBIbHixEEPO8IRqkY0",  # playlist?list=[]
        "title": "Goddestruktor",
        "thumbnails": thumbnails,
        "type": "Album",
        "artists": [],
        "year": "2022",
    },
    {
        "browseId": "MPREb_y6gMGnblVaF",
        "playlistId": "OLAK5uy_kUp9ShADLecN0e4s1xxjX0PEQ38sbqrqA",
        "title": "Face of Death (v2023)",
        "thumbnails": thumbnails,
        "type": "Single",
        "artists": [],
        "year": "2023",
    },
]


19